In [110]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [111]:
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [112]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [113]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///Resources/hawaii.sqlite")
conn = engine.connect()

In [114]:
# reflect an existing database into a new model
Base = automap_base()

# reflect the tables
Base.prepare(autoload_with=engine)

In [115]:
# View all of the classes that automap found
Base.classes.keys()

['measurement', 'station']

In [116]:
# Save references to each table
measurement = Base.classes.measurement
station = Base.classes.station

In [117]:
# Create our session (link) from Python to the DB
Session = Session(bind=engine)

# Exploratory Precipitation Analysis

In [118]:
# Find the most recent date in the data set.
most_recent_date = Session.query(func.max(measurement.date)).all()
for date in most_recent_date:
    print(f"Most Recent Date {date}")

Most Recent Date ('2017-08-23',)


In [119]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database. 
past_year_data = 

# Calculate the date one year from the last date in data set.
prev_year

# Perform a query to retrieve the data and precipitation scores
last_yr_data

# Save the query results as a Pandas DataFrame. Explicitly set the column names


# Sort the dataframe by date


# Use Pandas Plotting with Matplotlib to plot the data


SyntaxError: invalid syntax (2664879937.py, line 3)

In [ ]:
# Use Pandas to calculate the summary statistics for the precipitation data


# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number of stations in the dataset


In [ ]:
# Design a query to find the most active stations (i.e. which stations have the most rows?)
# List the stations and their counts in descending order.


In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.


In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram


# Close Session

In [ ]:
# Close Session
session.close()

NameError: name 'session' is not defined